In [1]:
import torch
from copy import deepcopy
from fireworks import Message
from fireworks.extensions import IgniteJunction
from fireworks.utils.exceptions import EndHyperparameterOptimization
from ignite.metrics import Metric
from ignite.exceptions import NotComputableError
from fireworks.extensions import LocalMemoryFactory, Experiment
from fireworks.extensions.training import default_training_closure, default_evaluation_closure
from itertools import combinations, count

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from nonlinear_regression_utils import NonlinearModel, get_data
from nonlinear_regression import base_loss, ModelSaverMetric

/usr/local/lib/python3.5/dist-packages/sqlalchemy/sql/functions.py:68: SAWarning: The GenericFunction 'array_agg' is already registered and is going to be overriden.
  "is going to be overriden.".format(identifier))
/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
train_set, test_set, params = get_data(n=1000)
loss = lambda batch: base_loss(batch['y_pred'], batch['y'])

In [3]:
# Specify hyperparameter optimization scheme using Factory
def make_model(parameters):
    temp_parameters = deepcopy(parameters)
    include = [letter for letter in ['a','b','c','d','e'] if letter in parameters]
    exclude = [letter for letter in ['a','b','c','d','e'] if letter not in parameters]
    for letter in exclude:
        temp_parameters[letter] =  [0]
    model = NonlinearModel(temp_parameters)
    for letter in exclude: # Prevent training from taking place for these parameters
        model.freeze(letter)
    return model

In [8]:
def get_trainer(train_set, loss, optimizer, **kwargs):

    def train_from_params(parameters):

        model = make_model(parameters)
        trainer = IgniteJunction(components={'model': model, 'dataset': train_set}, loss=loss, optimizer=optimizer, visdom=False,**kwargs)
        print("Now training model for parameters {0}".format(parameters))
        trainer.train(max_epochs=10)
        evaluator = IgniteJunction(components={'model': model, 'dataset': train_set}, loss=loss, optimizer=optimizer, update_function=default_evaluation_closure, visdom=False, **kwargs)
        print("Now evaluating trained model.")
        return trainer

    return train_from_params

In [9]:
class Parameterizer:

    def __init__(self):
        possible_params = ['a','b','c','d','e']
        def generator():
            for i in reversed(range(5)):
                for combination in combinations(possible_params,i):
                    params = {param: [0] for param in combination}
                    yield params
        self.generator = generator()

    def __call__(self,past_params, metrics):
        try:
            params = self.generator.__next__()
            if params == {}:
                raise EndHyperparameterOptimization
            return params

        except StopIteration:
            raise EndHyperparameterOptimization

In [10]:
class AccuracyMetric(Metric):

    def __init__(self, output_transform = lambda x:x):
        Metric.__init__(self, output_transform=output_transform)
        self.reset()

    def reset(self):
        self.l2 = 0.
        self.num_examples = 0

    def update(self, output):
        self.l2 += output['loss']
        self.num_examples += len(output['output'])

    def compute(self):

        if self.num_examples == 0:
            raise NotComputableError(
                "Metric must have at least one example before it can be computed."
            )
        return Message({'average-loss': [self.l2 / self.num_examples]}).to_dataframe()

In [11]:
description = "In this experiment, we will compare the performance of different polynomial models when regressed against data generated from a random polynomial."
experiment = Experiment("model_selection", description=description)

factory = LocalMemoryFactory(components={
    'trainer': get_trainer(train_set, loss, optimizer='Adam', lr=.1),
    'eval_set': test_set,
    'parameterizer': Parameterizer(),
    'metrics': {'accuracy': AccuracyMetric(), 'model_state': ModelSaverMetric()}
    })

factory.run()

# Plot the different runs
fig, ax = plt.subplots()
model = NonlinearModel()
true_model = NonlinearModel(components={'a':[params['a']], 'b': [params['b']], 'c': [params['c']], 'd': [0], 'e': [0]})
x = Message({'x':np.arange(-10,10,.2)}).to_tensors()
y_true = true_model(x)['y_pred'].detach().numpy()

Now training model for parameters {'c': [0], 'b': [0], 'd': [0], 'a': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 93476944.00
Epoch[4] Iteration: 100 Time: 0:00:00 Loss: 784026.06
Epoch[7] Iteration: 200 Time: 0:00:01 Loss: 22.97
Now evaluating trained model.
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 49.58
Epoch[1] Iteration: 100 Time: 0:00:00 Loss: 18035524.00
Epoch[1] Iteration: 200 Time: 0:00:01 Loss: 0.62
Now training model for parameters {'c': [0], 'b': [0], 'e': [0], 'a': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 41441532.00
Epoch[4] Iteration: 100 Time: 0:00:00 Loss: 5002926.00
Epoch[7] Iteration: 200 Time: 0:00:01 Loss: 5248909.00
Now evaluating trained model.
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 15545025.00
Epoch[1] Iteration: 100 Time: 0:00:00 Loss: 10927634432.00
Epoch[1] Iteration: 200 Time: 0:00:01 Loss: 26.24


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Now training model for parameters {'e': [0], 'b': [0], 'd': [0], 'a': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 72360688.00
Epoch[4] Iteration: 100 Time: 0:00:00 Loss: 4317777.50
Epoch[7] Iteration: 200 Time: 0:00:01 Loss: 7685912.00
Now evaluating trained model.
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 9532795.00
Epoch[1] Iteration: 100 Time: 0:00:00 Loss: 1500968320.00
Epoch[1] Iteration: 200 Time: 0:00:01 Loss: 48.93
Now training model for parameters {'c': [0], 'e': [0], 'd': [0], 'a': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 69434944.00
Epoch[4] Iteration: 100 Time: 0:00:00 Loss: 6635392.50
Epoch[7] Iteration: 200 Time: 0:00:01 Loss: 8854227.00
Now evaluating trained model.
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 7085496.50
Epoch[1] Iteration: 100 Time: 0:00:00 Loss: 58307600.00
Epoch[1] Iteration: 200 Time: 0:00:01 Loss: 59.40
Now training model for parameters {'c': [0], 'b': [0], 'd': [0], 'e': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 32441476.00
Epoch[4] Iter

Epoch[1] Iteration: 200 Time: 0:00:01 Loss: 90.41
Now training model for parameters {'c': [0], 'd': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 47676796.00
Epoch[4] Iteration: 100 Time: 0:00:00 Loss: 289231.53
Epoch[7] Iteration: 200 Time: 0:00:01 Loss: 323.95
Now evaluating trained model.
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 28.60
Epoch[1] Iteration: 100 Time: 0:00:00 Loss: 28666644.00
Epoch[1] Iteration: 200 Time: 0:00:01 Loss: 64.86
Now training model for parameters {'c': [0], 'e': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 57047324.00
Epoch[4] Iteration: 100 Time: 0:00:00 Loss: 10110506.00
Epoch[7] Iteration: 200 Time: 0:00:01 Loss: 24867988.00
Now evaluating trained model.
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 7469219.50
Epoch[1] Iteration: 100 Time: 0:00:00 Loss: 962187840.00
Epoch[1] Iteration: 200 Time: 0:00:01 Loss: 80.74
Now training model for parameters {'d': [0], 'e': [0]}
Epoch[1] Iteration: 0 Time: 0:00:00 Loss: 59962500.00
Epoch[4] Iteration: 100 Time: 0:

In [14]:
# Now we can plot the results

def animate(frame):

    current_state = {'internal': frame[1]['internal'][0], 'external': {}}
    model.set_state(current_state)

    y_predicted = model(x)['y_pred'].detach().numpy()
    xdata = list(x['x'].detach().numpy())
    ydata = list(y_predicted)
    ax.clear()
    ax.plot(xdata, list(y_true), 'r')
    ax.plot(xdata, ydata, 'g')

    # Set up titles
    substrings = []
    if (frame[0]['a'] == 0).all():
        substrings.append("a")
    if (frame[0]['b'] == 0).all():
        substrings.append("bx")
    for key, i in zip(['c','d','e'], count()):
        if (frame[0][key] == 0).all():
            substrings.append("{0}x^{1}".format(key,i+2))
    title = "Model: {0}".format(" + ".join(substrings))

    ax.set_title(title)

params, metrics = factory.read()
accuracy_file = experiment.open('accuracy.csv', string_only=True)
metrics['accuracy'].to('csv', path=accuracy_file)
model_state_file = experiment.open('model_states.csv', string_only=True)
metrics['model_state'].to('csv', path=model_state_file)
params_file = experiment.open('params.csv', string_only=True)
params.to('csv', path=params_file)

print(params)


Message with 
 Tensors: 
 TensorMessage: {} 
 Metadata: 
       a    b    c    d    e
0   0.0  0.0  0.0  0.0  NaN
1   0.0  0.0  0.0  NaN  0.0
2   0.0  0.0  NaN  0.0  0.0
3   0.0  NaN  0.0  0.0  0.0
4   NaN  0.0  0.0  0.0  0.0
5   0.0  0.0  0.0  NaN  NaN
6   0.0  0.0  NaN  0.0  NaN
7   0.0  0.0  NaN  NaN  0.0
8   0.0  NaN  0.0  0.0  NaN
9   0.0  NaN  0.0  NaN  0.0
10  0.0  NaN  NaN  0.0  0.0
11  NaN  0.0  0.0  0.0  NaN
12  NaN  0.0  0.0  NaN  0.0
13  NaN  0.0  NaN  0.0  0.0
14  NaN  NaN  0.0  0.0  0.0
15  0.0  0.0  NaN  NaN  NaN
16  0.0  NaN  0.0  NaN  NaN
17  0.0  NaN  NaN  0.0  NaN
18  0.0  NaN  NaN  NaN  0.0
19  NaN  0.0  0.0  NaN  NaN
20  NaN  0.0  NaN  0.0  NaN
21  NaN  0.0  NaN  NaN  0.0
22  NaN  NaN  0.0  0.0  NaN
23  NaN  NaN  0.0  NaN  0.0
24  NaN  NaN  NaN  0.0  0.0
25  0.0  NaN  NaN  NaN  NaN
26  NaN  0.0  NaN  NaN  NaN
27  NaN  NaN  0.0  NaN  NaN
28  NaN  NaN  NaN  0.0  NaN
29  NaN  NaN  NaN  NaN  0.0


In [15]:
print(metrics)

defaultdict(<class 'fireworks.core.message.Message'>, {'accuracy': Message with 
 Tensors: 
 TensorMessage: {} 
 Metadata: 
     average-loss
0   1.283122e+07
1   1.463471e+10
2   4.522753e+10
3   5.317224e+10
4   3.488390e+10
5   1.013811e+01
6   1.107777e+08
7   8.674417e+10
8   1.344326e+07
9   3.713649e+10
10  5.777195e+10
11  1.334244e+07
12  8.342510e+10
13  9.584667e+10
14  8.969755e+10
15  6.337022e+07
16  3.756494e-01
17  1.127991e+08
18  8.732904e+10
19  2.891355e+01
20  1.030167e+08
21  9.760181e+10
22  1.244828e+07
23  3.313519e+10
24  6.114724e+10
25  6.331644e+07
26  6.385987e+07
27  2.810616e+01
28  1.074338e+08
29  8.597056e+10, 'model_state': Message with 
 Tensors: 
 TensorMessage: {} 
 Metadata: 
    external                                           internal  iteration
0        {}  {'c': [7.019401], 'training': True, 'b': [-0.4...        200
1        {}  {'c': [1.0177288], 'training': True, 'b': [2.3...        200
2        {}  {'c': [0.0], 'training': True, 'b': [1.

In [17]:
ani = FuncAnimation(fig, animate, zip(factory.params, factory.computed_metrics['model_state']), interval=3000)
ani.save(experiment.open("models.mp4", string_only=True)) # This will only work if you have ffmpeg installed.
plt.show()
